In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.2 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.2 which is incompatible.
tensorboardx 2.5.1 requires protobuf<=3.20.1,>=3.8.0, but you have protobuf 3.20.2 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [2]:
!nvidia-smi

Wed May  3 20:15:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Pretrained Pegasus Model

In [24]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [25]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [26]:
model=model_pegasus

# Loading Dataset

In [5]:
df=pd.read_csv("/kaggle/input/chat-gpt-dataset/dataset.csv")

In [6]:
df

,Unnamed: 0,x,y
0,0,"Ahead of AyodhyaVerdict, appeal to everyone to...",welcome Hon Supreme Court's AyodhyaVerdict on...
1,1,"WATCH Maulana Asghar Ali Salafi, President, Ma...",is a landmark decision by the Supreme Court o...
2,2,For all those of the western media that were g...,\n\nThis group of tweets is discussing the rea...
3,3,Reporter's DiaryMet two kids who sell diyas on...,".\n\nThese tweets discuss the Ayodhya verdict,..."
4,4,Just bcoz BJP got 303 seats in LS everything l...,: This tweet is suggesting that the BJP's succ...
...,...,...,...
1036,1036,CNN nbcsnl joerogan WHO WoodyHarrelson COVID19...,\n\nThese tweets discuss the COVID-19 pandemic...
1037,1037,Maddow Bitcoin CNN BernieSanders ElonMusk Eliz...,\n\nThese tweets discuss the ongoing COVID-19 ...
1038,1038,:red_circle: FBI Director COVID19 Pandemic 'l...,\n\nThese tweets are discussing the origin of ...
1039,1039,GOP Sen. Tom Cotton RAR slammed liberal media ...,\nSen. Tom Cotton has criticized the liberal m...


# Train Test Split

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=1)

In [8]:
test['x'][148]

'Contact for Learning Risk Management, Options Selling amp How to catch Market trends.:pushpin:Nifty StockMarketNifty50 NiftyBankAdaniGroup AdaniAdaniEnterprisesstockmarketcrash Telegram channel joining link.:red_triangle_pointed_down: Morning Minutes✦Group Moves Reliance, Adani amp Jindal group amp HDFC twins trade in green, while Tata group trade mixed✦Stocks in focus PowerGrid, ZydusLifesciences, TataPower, VodafoneIdea, Seamec, AdaniEntStockMarket nseindia BSEHindenburgRes BSDK Tool kit of georgesoros and INCIndia trying to demolish Indian Democracy by attacking Adani . We know everything who is behind this. Wait n watch for reaction."Coincidentally, Adani Powers sister concern Adani Enterprises is Indias biggest coal trader and owns mines in Indonesia and Australia – and the coal for the Godda plant will most likely be supplied by that vendor." StopAdani MSCIInc  TRANSMISSION Rs 27.23 crores NSE Block Trade for  4,04,617 shares, at Rs 673blockdeals nseindiaWho is buying Adani shar

In [9]:
test['y'][148]

"\n\nThis tweet thread is about the controversies surrounding Adani Enterprises, a coal trader and mine owner in India. It discusses how the Modi government has allegedly given special favors to Adani, such as allowing them to mine from a block with a large amount of coal. It also mentions how the Adani and Vedanta debacles have caused a decline in stock prices and investor caution. Finally, it talks about the LPG price hike to compensate for Adani's losses."

# Evaluating Pegasus pretrained model on my dataset

In [32]:
pipe = pipeline('summarization', model = model_ckpt)

In [33]:
pipe_out = pipe(test['x'][148])
print(pipe_out)

[{'summary_text': 'Reliance, Adani amp Jindal group amp twins trade in green, while Tata group trade mixedStocks in focus PowerGrid, ZydusLifesciences, TataPower, VodafoneIdea, Seamec, AdaniEntStockMarket nse India .'}]


# Prediction

In [10]:
def get_response(input_text):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=1024, return_tensors="pt").to(device)
  gen_out = model.generate(**batch,max_length=150,num_beams=5, num_return_sequences=1, temperature=1.5)
  output_text = tokenizer.batch_decode(gen_out, skip_special_tokens=True)
  return output_text

In [35]:
get_response(test['x'][148])

['Reliance, Adani amp Jindal group amp twins trade in green, while Tata group trade mixedStocks in focus PowerGrid, ZydusLifesciences, TataPower, VodafoneIdea, Seamec, AdaniEntStockMarket nse India.']

In [36]:
pred = []
for i in tqdm(test['x'], total=len(test)):
    pred.append(get_response(i))
    

100%|██████████| 105/105 [04:58<00:00,  2.84s/it]


In [37]:
ref_text=[]
for i in tqdm(test['y'], total=len(test)):
    ref_text.append(i)

100%|██████████| 105/105 [00:00<00:00, 259212.43it/s]


## Calculating rogue score on pre-trained model

In [38]:
pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [39]:
from datasets import load_dataset, load_metric
rouge_metric = load_metric('rouge')
rouge_metric.add_batch(predictions=pred, references=ref_text)
        
#  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.260266,0.111055,0.197295,0.196993


# Fine Tuning the Model

In [42]:
from datasets import Dataset
train_ds=Dataset.from_pandas(train)
test_ds=Dataset.from_pandas(test)

In [43]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['x'] , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['y'], max_length = 150, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
train_pt = train_ds.map(convert_examples_to_features, batched = True)
test_pt=test_ds.map(convert_examples_to_features, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3587: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

In [44]:
len(train_pt['input_ids'][0])

424

In [45]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [46]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus/cnn-daily', num_train_epochs=20, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [47]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train_pt, 
                  eval_dataset=test_pt)

In [48]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.435800,1.426937
1000,1.043500,1.391124


TrainOutput(global_step=1160, training_loss=1.573173005827542, metrics={'train_runtime': 6577.2525, 'train_samples_per_second': 2.846, 'train_steps_per_second': 0.176, 'total_flos': 2.0856805964292096e+16, 'train_loss': 1.573173005827542, 'epoch': 19.83})

# Saving the Trained model

In [49]:
trainer.save_model("/kaggle/working/")

In [50]:
model_pegasus.save_pretrained("finetuned-model")

In [51]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

# Testing on the trained model

In [11]:
model_ckpt = "/kaggle/input/pegasus-fine-tuned-final"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [12]:
pipe = pipeline('summarization', model = model_ckpt)

In [17]:
test['x'][148]

'Contact for Learning Risk Management, Options Selling amp How to catch Market trends.:pushpin:Nifty StockMarketNifty50 NiftyBankAdaniGroup AdaniAdaniEnterprisesstockmarketcrash Telegram channel joining link.:red_triangle_pointed_down: Morning Minutes✦Group Moves Reliance, Adani amp Jindal group amp HDFC twins trade in green, while Tata group trade mixed✦Stocks in focus PowerGrid, ZydusLifesciences, TataPower, VodafoneIdea, Seamec, AdaniEntStockMarket nseindia BSEHindenburgRes BSDK Tool kit of georgesoros and INCIndia trying to demolish Indian Democracy by attacking Adani . We know everything who is behind this. Wait n watch for reaction."Coincidentally, Adani Powers sister concern Adani Enterprises is Indias biggest coal trader and owns mines in Indonesia and Australia – and the coal for the Godda plant will most likely be supplied by that vendor." StopAdani MSCIInc  TRANSMISSION Rs 27.23 crores NSE Block Trade for  4,04,617 shares, at Rs 673blockdeals nseindiaWho is buying Adani shar

In [18]:
pipe_out = pipe(test['x'][148])
print(pipe_out)

[{'summary_text': 'Reliance, Adani, Jindal group, HDFC twins trade in green, while Tata group trade mixed. Stocks in focus PowerGrid, ZydusLifesciences, TataPower, VodafoneIdea, Seamec,. AdaniEntStockMarket nseindia BSEHindenburgRes BSDK Tool kit of georgesoros and INCIndia trying to demolish Indian Democracy by attacking Adani . We know everything who is behind this. Wait n watch for reaction."'}]


In [13]:
model = model_pegasus

In [14]:
pred = []
for i in tqdm(test['x'], total=len(test)):
    pred.append(get_response(i))

100%|██████████| 105/105 [02:55<00:00,  1.67s/it]


In [15]:
ref_text=[]
for i in tqdm(test['y'], total=len(test)):
    ref_text.append(i)

100%|██████████| 105/105 [00:00<00:00, 102371.44it/s]


In [16]:
from datasets import load_dataset, load_metric
rouge_metric = load_metric('rouge')
rouge_metric.add_batch(predictions=pred, references=ref_text)
        
#  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.36475,0.161847,0.252668,0.253634
